In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "classify-to-learn"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.v2 import ExtendedEncoder, Encoder

In [4]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split

trainL = ImageDataset(dataset=datasets.TRAINING_LABELED,network_input_size=256, cuda=cuda)
trainU = ImageDataset(dataset=datasets.TRAINING_UNLABELED,network_input_size=256, cuda=cuda)
#trainC = ConcatDataset([trainL,trainU])
trainC = trainL #doing it properly... it is necessary to do proper validation that the lavidation data are unseen during training of the model...
train_size = int(0.8*len(trainC))
val_size = len(trainC)-train_size
train,val = random_split(trainC,[train_size,val_size])
test = ImageDataset(dataset=datasets.TEST,network_input_size=256,cuda=cuda)


trainL = DataLoader(trainL,128,True)
train = DataLoader(train,128,True)
val = DataLoader(val,128,True)
test = DataLoader(test,128,True)

In [5]:
import csv
translate_label = []

# Read the CSV file
with open('disambiguation.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        label = row[1]
        # Ensure the list is large enough to hold the current index
        
        translate_label.append(label)

print(translate_label)


['macaron', 'bignet', 'cruller', 'vongole_clams', 'triangoli_fritti', 'tiramisu', 'mexican_tostadas', 'lamb_moussaka', 'ravioli_cinesi', 'sushi_sashimi', 'knishes', 'crocchette', 'couscous_verdure', 'porridge', 'lattuga_kimchi', 'alga_wakame', 'spaghetti_squash', 'pasta_rigatoni', 'tartare_di_carne', 'cannolo_siciliano', 'fegato_oca', 'cupcakes', 'carne_ossobuco', 'tagliatelle_con_frutti_di_mare', 'canadian_poutine_w_mushroom_gravy', 'ramen', 'panino_pulled_pork', 'ramen_con_uovo_in_mezzo', 'chicken_kiev', 'torta_mele_stile_crostata', 'risotto', 'torta_stile_torrone_canditi', 'maccheroni_con_carne', 'gluten_free_haggis', 'uova_strapazzate_colazione', 'frittata', 'shrimp_scampi', 'sushi_maki', 'orzo_bolognese', 'amish_apple_fritters', 'nachos', 'beef_stroganoff', 'beef_wellington', 'involtino_primavera', 'dolce_rotondo_con_buco', 'gamberoni', 'souffle', 'maiale_cinese', 'muffin', 'uovo_sodo_con_mousse', 'escargots', 'tramezzino_pollo_club_sandwich', 'torta_di_carote', 'falafell', 'farfa

In [6]:

device = torch.device('cpu')
PATH = "Storage\models\AutoEncoderModelV2.pth"
if cuda:
    model = torch.load(PATH)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
else:    
    model = torch.load(PATH, map_location=torch.device('cpu') )


In [10]:
from sklearn.metrics import confusion_matrix, top_k_accuracy_score

all_preds = []
all_labels = []
all_out = []
model.eval()
with torch.no_grad():
    for inputs,labels in tqdm(test):
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        all_out.extend(outputs.cpu().numpy())
        
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_out = np.array(all_out)
cm = confusion_matrix(all_labels, all_preds)


100%|██████████| 94/94 [04:10<00:00,  2.67s/it]


In [12]:
correct = (all_preds==all_labels).sum().item()
print(correct)
print(len(ImageDataset(dataset=datasets.TEST,network_input_size=256,cuda=cuda)))
print(f'Accuracy: {100 * correct / len(ImageDataset(dataset=datasets.TEST,network_input_size=256,cuda=cuda))}%') #TOO BAD!

75
11994
Accuracy: 0.6253126563281641%
